In [1]:
import torch
from transformers import BertJapaneseTokenizer, BertModel

MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
model = BertModel.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
sentences1 = ["The cat is sleeping.", "The dog is running.",
              "The mouse is hiding.", "The fish is swimming.", "The bird is singing."]
sentences2 = ["The dog is barking.", "The cat is playing.",
              "The bird is flying.", "The fish is jumping.", "The mouse is eating."]

In [4]:
def sentence_to_vector(model, tokenizer, sentence):
    tokens = tokenizer(sentence, add_special_tokens=True)["input_ids"]
    input = torch.tensor(tokens).unsqueeze(0)
    with torch.no_grad():
        outputs = model(input, output_hidden_states=True)
        last_hidden_state = outputs[0][:, 0, :]
        averaged_hidden_state = last_hidden_state.mean(dim=0).unsqueeze(0)
    return averaged_hidden_state

In [11]:
def calc_similarity(sentence1, sentence2):
    print("{}\n{}".format(sentence1, sentence2))

    sentence_vector1 = sentence_to_vector(model, tokenizer, sentence1)
    sentence_vector2 = sentence_to_vector(model, tokenizer, sentence2)

    # Reshape the tensors to 1D
    sentence_vector1 = sentence_vector1.reshape(-1)
    sentence_vector2 = sentence_vector2.reshape(-1)

    # Calculate the cosine similarity
    similarity = float(torch.nn.functional.cosine_similarity(
        sentence_vector1, sentence_vector2, dim=0).detach().numpy().copy())
    print("Similarity:", similarity)

    return similarity

In [12]:
def calc_average_similarity(sentences):
    similarities = []
    for i in range(1, len(sentences)):
        similarity = calc_similarity(sentences[0], sentences[i])
        similarities.append(similarity)
    average_similarity = sum(similarities) / len(similarities)
    print("Similarities:", similarities)
    print("Average similarity:", average_similarity)

    return average_similarity

In [14]:
calc_average_similarity(sentences1)

The cat is sleeping.
The dog is running.
Similarity: 0.955855131149292
The cat is sleeping.
The mouse is hiding.
Similarity: 0.9288669228553772
The cat is sleeping.
The fish is swimming.
Similarity: 0.9353625774383545
The cat is sleeping.
The bird is singing.
Similarity: 0.9280245900154114
Similarities: [0.955855131149292, 0.9288669228553772, 0.9353625774383545, 0.9280245900154114]
Average similarity: 0.9370273053646088


0.9370273053646088